## **Title: "Enhancing US Address Data with State, County, and Country Information using GPT-4 and Visualizing with Folium"**

**Objective:**
* The objective of this project is to find the place , county(the largest political division of a state in the US) and country of given dataset and this is done through implementing "gpt-4"  AI module and along with it we also implemented the library folium to visualize the area (or) a specific locations of the companies.

In [ ]:
# Import Necessary Libraries
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import pandas as pd
import os

# Load dataset
file_path = "E:\\Projects\\Datasets\\US_addresses.xlsx"  
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name="Sheet1")

# Set up OpenAI API Key
os.environ["OPENAI_API_KEY"] = "Replace with actual API key"

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4-turbo")

# Apply the function
def get_location_details(address, city, pincode):
    """Use GPT-4 to find State, County, and Country based on Address."""
    prompt = f"""
    Given the following address details:
    Address: {address}
    City: {city}
    Pincode: {pincode}
    
    Provide the corresponding State, County, and Country. Format the output as:
    State: <state_name>
    County: <county_name>
    Country: <country_name>
    """
    
    response = llm([HumanMessage(content=prompt)])
    
    # Extract details from response
    try:
        lines = response.content.split("\n")
        state = lines[0].split(": ")[1]
        county = lines[1].split(": ")[1]
        country = lines[2].split(": ")[1]
        return state, county, country
    except:
        return None, None, None

# Process dataset
for index, row in df.iterrows():
    state, county, country = get_location_details(row["Street Name"], row["City"], row["Pincode"])
    df.at[index, "State"] = state
    df.at[index, "County"] = county
    df.at[index, "Countery"] = country
    df.at[index, "Place"] = f"{row['City']}, {state}, {country}" if state and country else "Unknown"

# Save updated dataset
df.to_excel("Updated_US_addresses.xlsx", index=False)

print("Dataset updated successfully!")

C:\Users\gkris\AppData\Local\Temp\ipykernel_8864\1855127479.py:31: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt)])
C:\Users\gkris\AppData\Local\Temp\ipykernel_8864\1855127479.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tennessee' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, "State"] = state
C:\Users\gkris\AppData\Local\Temp\ipykernel_8864\1855127479.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Williamson' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, "County"] = county
C:\Users\gkris\AppData\Local\Temp\ipykernel_8864\1855127479.py:4

Dataset updated successfully!


In [43]:
# Check the dataset
df.head()

,Company Name,Street Name,Place,City,Pincode,State,County,Countery
0,Wipro Limited,"810 Crescent Centre Drive, Suite 400","Franklin, Tennessee, United States",Franklin,TN 37067,Tennessee,Williamson,United States
1,Wipro Limited,"888 W. Big Beaver Road, Suite 1200","Troy, Michigan, United States",Troy,MI 48084,Michigan,Oakland,United States
2,Wipro Limited,"2 Tower Center Boulevard, Suite 2200","East Brunswick, New Jersey, United States",East Brunswick,NJ 08816,New Jersey,Middlesex,United States
3,Wipro Limited,"1114 Avenue of the Americas, Suite 3030","New York, New York, United States",New York,NY 10036,New York,New York,United States
4,Wipro Limited,"2858 Woodcock Blvd, Suites 100-300","Atlanta, Georgia, United States",Atlanta,GA 30341,Georgia,DeKalb,United States


In [45]:
# For Visualization import geopy
!pip install geopy

In [ ]:
# Visualize it through folium 
import folium
from geopy.geocoders import Nominatim

# Create a map centered around the United States
map_center = [37.0902, -95.7129]  # Latitude and Longitude of the center of the US
mymap = folium.Map(location=map_center, zoom_start=4)

# Add markers for each address
for index, row in df.iterrows():
    address = row["Street Name"]
    city = row["City"]
    state = row["State"]
    country = row["Countery"]
    place = row["Place"]
    
    # Use geopy to get the latitude and longitude of the address
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(f"{address}, {city}, {state}, {country}")
    
    if location:
        folium.Marker(
            location=[location.latitude, location.longitude],
            popup=place,
            tooltip=address
        ).add_to(mymap)

# Save the map to an HTML file
mymap.save("US_addresses_map.html")

# Display the map
mymap